In [1]:
from main import *
import cluster
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import datetime

def format_time(s):
    dtf = datetime.time.fromisoformat("00:0" + s.strip())
    return dtf.second + dtf.microsecond/1_000_000

In [2]:
nb_meas = 8  # this is how many measures were taken (i.e. how many measurement columns are in the dataframe)
perf_matrix, nb_data = load_all_csv("../data/res_ugc/", ext="csv", with_names=True)
idx = compute_index(perf_matrix, nb_data)
data_per_cfg = sort_data(perf_matrix, idx, nb_data)
measures = cluster.extract_feature(data=data_per_cfg, nb_meas=nb_meas+1).iloc[:, :nb_meas]
measure_names = list(measures.columns.values)

In [3]:
input_config_map = data_per_cfg[["inputname", "configurationID"] + measure_names].sort_values(["inputname", "configurationID"])
input_config_map["elapsedtime"] = input_config_map["elapsedtime"].apply(format_time)
all_input_names = sorted(input_config_map.inputname.unique())
all_config_ids = sorted(input_config_map.configurationID.unique())
len(all_input_names), len(all_config_ids)

(1397, 201)

In [4]:
def input_distance(input1, input2):
    i1 = input_config_map[input_config_map["inputname"] == all_input_names[input1]][measure_names]
    i2 = input_config_map[input_config_map["inputname"] == all_input_names[input2]][measure_names]
    return np.abs(i1.values - i2.values).mean()

input_distance(0, 1)

3788463.0577985076

In [5]:
X_inps = np.empty((len(all_input_names), len(all_input_names)))

for i1 in range(len(all_input_names)):
    for i2 in range(i1+1, len(all_input_names)):
        X_inps[i1, i2] = input_distance(i1, i2)
        X_inps[i2, i1] = X_inps[i1, i2]

KeyboardInterrupt: 

In [ ]:
i2

,size,usertime,systemtime,elapsedtime,cpu,frames,fps,kbs
0,64636599,94.76,11.80,0:23.88,446,1199,64.08,25876.20
111,71046144,168.19,11.77,0:27.74,648,1199,53.47,28442.16
123,35371666,213.73,13.42,0:30.00,757,1199,49.42,14160.47
134,37436916,262.58,13.69,0:30.95,892,1199,46.62,14987.26
145,46497463,296.50,12.84,0:32.74,944,1199,44.36,18614.50
...,...,...,...,...,...,...,...,...
106,45721811,6990.67,64.61,3:21.60,3499,1199,6.10,18303.98
107,42554890,2827.79,63.45,2:28.24,1950,1199,8.37,17036.15
108,42347309,7894.44,63.51,3:27.12,3842,1199,5.93,16953.05
109,42347309,8123.67,68.01,3:29.07,3917,1199,5.89,16953.05


In [ ]:
configurations = data_per_cfg.iloc[:,:-nb_meas].drop_duplicates()

In [ ]:
data_per_cfg[["configurationID", "inputname"]]

,configurationID,inputname
0,1,NewsClip_480P-6bfa
0,1,Gaming_2160P-4258
0,1,Animation_2160P-6f3b
0,1,Animation_1080P-2fff
0,1,Sports_480P-41a5
...,...,...
200,10,Animation_360P-188f
200,10,Animation_1080P-4214
200,10,Animation_480P-7a31
200,10,Sports_1080P-0d0c


In [ ]:
measures.iloc[:, :nb_meas]

,size,usertime,systemtime,elapsedtime,cpu,frames,fps,kbs
0,11103653,1.82,0.20,0:00.36,555,499,2169.41,4450.36
0,64636599,94.76,11.80,0:23.88,446,1199,64.08,25876.20
0,51361891,32.51,4.54,0:07.58,488,479,82.03,20587.65
0,146517606,17.39,1.30,0:02.18,856,499,309.09,58724.49
0,35051981,2.54,0.24,0:00.49,560,500,1460.46,14020.79
...,...,...,...,...,...,...,...,...
200,1240174,37.26,1.13,0:06.20,619,599,98.86,496.40
200,7547458,563.79,6.91,0:25.55,2233,500,20.03,3018.98
200,2635249,44.68,0.33,0:04.78,940,158,33.68,1067.44
200,35388899,1634.93,14.79,1:13.28,2251,1200,16.66,14155.56


In [ ]:
index_interest = [0, 4]
feature_pts = cluster.create_feature_points(measures, nb_data, index_interest)
feature_pts

,0,1,2,3,4,5,6,7,8,9,...,2784,2785,2786,2787,2788,2789,2790,2791,2792,2793
0,1.82,499.0,94.76,1199.0,32.51,479.0,17.39,499.0,2.54,500.0,...,1.01,599.0,8.05,500.0,0.68,158.0,27.86,1200.0,23.96,1200.0
1,9.74,499.0,334.50,1199.0,120.78,479.0,75.82,499.0,17.06,500.0,...,3.87,599.0,26.33,500.0,2.72,158.0,92.51,1200.0,82.59,1200.0
2,9.59,499.0,269.71,1199.0,103.96,479.0,81.50,499.0,16.71,500.0,...,3.41,599.0,22.00,500.0,2.85,158.0,92.54,1200.0,69.27,1200.0
3,11.32,499.0,314.03,1199.0,120.92,479.0,92.62,499.0,19.88,500.0,...,3.71,599.0,26.83,500.0,3.02,158.0,104.12,1200.0,86.54,1200.0
4,13.72,499.0,915.69,1199.0,224.45,479.0,137.70,499.0,23.33,500.0,...,4.17,599.0,35.58,500.0,3.76,158.0,165.41,1200.0,149.37,1200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,7.23,499.0,329.17,1199.0,127.91,479.0,51.65,499.0,11.33,500.0,...,3.62,599.0,19.49,500.0,1.93,158.0,65.20,1200.0,68.83,1200.0
197,5.80,499.0,265.16,1199.0,102.59,479.0,45.65,499.0,10.56,500.0,...,2.57,599.0,18.96,500.0,1.79,158.0,56.94,1200.0,58.37,1200.0
198,9.82,499.0,445.12,1199.0,154.65,479.0,80.12,499.0,17.30,500.0,...,3.70,599.0,30.24,500.0,2.82,158.0,104.90,1200.0,93.71,1200.0
199,8.11,499.0,391.95,1199.0,126.00,479.0,84.41,499.0,14.88,500.0,...,3.07,599.0,28.09,500.0,3.38,158.0,99.89,1200.0,76.92,1200.0


In [ ]:
def mymetric(a, b=0):
    print(a.shape)
    print(b)
    return np.sqrt(a-b)

model = AgglomerativeClustering(n_clusters=None, metric=mymetric, distance_threshold=0, compute_distances=True, linkage='average')
model.fit(feature_pts)

(201, 2794)
0


AgglomerativeClustering(compute_distances=True, distance_threshold=0,
                        linkage='average',
                        metric=<function mymetric at 0x177b6e340>,
                        n_clusters=None)

In [ ]:
from sklearn.metrics import pairwise_distances
xxx = pairwise_distances(feature_pts)


In [ ]:
xxx.shape

(201, 201)